In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.pyplot import figure
import pyodbc 
import glob

In [2]:
# Must include MGRA
columns_to_check = ['mgra', 'pop', 'hhp', 'gq_civ', 'gq_mil']

In [3]:
config_filename = 'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/ds41_42_config.yml'

In [4]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [5]:
DS38_datafiles = ['DS38_2016', 'DS38_2018', 'DS38_2020', 'DS38_2023', 'DS38_2025', 'DS38_2026', 'DS38_2029', 'DS38_2030', 'DS38_2032', 'DS38_2035', 'DS38_2040', 'DS38_2045', 'DS38_2050']

In [6]:
DS42_datafiles = ['DS42_2016', 'DS42_2018', 'DS42_2020', 'DS42_2023', 'DS42_2025', 'DS42_2026', 'DS42_2029', 'DS42_2030', 'DS42_2032', 'DS42_2035', 'DS42_2040', 'DS42_2045', 'DS42_2050']

In [7]:
older_DS = 'DS38'
current_DS = 'DS42'

# Creating the MGRA level Data Frame

In [8]:
# Older Data Frame
DS38_Data = pd.DataFrame(columns = columns_to_check)
for file_name in DS38_datafiles:
    working_df = pd.read_csv(config[older_DS][file_name], usecols=columns_to_check)
    working_df['year'] = f"{file_name[-4:]}"
    DS38_Data = DS38_Data.append(working_df)

In [9]:
#Making it original 
DS38_Data.columns = [x + '_DS38' for x in DS38_Data.columns]

In [10]:
# Newer Data Frame
DS42_Data = pd.DataFrame(columns = columns_to_check)
for file_name in DS42_datafiles:
    working_df = pd.read_csv(config[current_DS][file_name], usecols=columns_to_check)
    working_df['year'] = f"{file_name[-4:]}"
    DS42_Data = DS42_Data.append(working_df)

In [11]:
# making it original
DS42_Data.columns = [x + '_DS42' for x in DS42_Data.columns]

In [12]:
# Combine to make one Data Frame
mgra_df = pd.concat([DS38_Data, DS42_Data], axis=1)

In [13]:
# Clean up MGRA DF
mgra_df = mgra_df.rename(columns={'mgra_DS38':'mgra'})
mgra_df = mgra_df.drop(['mgra_DS42', 'year_DS38'], axis=1)
mgra_df = mgra_df.groupby(['mgra', 'year_DS42']).sum()

In [14]:
mgra_df

pop_DS38  hhp_DS38  gq_civ_DS38  gq_mil_DS38  pop_DS42  \
mgra  year_DS42                                                           
1     2016             41        41            0            0        41   
      2018             41        41            0            0        44   
      2020             47        47            0            0        44   
      2023             38        38            0            0        46   
      2025             35        35            0            0        45   
...                   ...       ...          ...          ...       ...   
23002 2032              0         0            0            0         0   
      2035              0         0            0            0         0   
      2040              0         0            0            0         0   
      2045              0         0            0            0         0   
      2050              0         0            0            0         0   

                 hhp_DS42  gq_civ_DS42  gq_mil_DS42  
mgra  year_DS42                                      
1     2016             41            0            0  
      2018             44            0            0  
      2020             44            0            0  
      2023             46            0            0  
      2025             45            0            0  
...                   ...          ...          ...  
23002 2032              0            0            0  
      2035              0            0            0  
      2040              0            0            0  
      2045              0            0            0  
      2050              0            0            0  

[299026 rows x 8 columns]

In [15]:
hhp_diff = (mgra_df['hhp_DS42'] - mgra_df['hhp_DS38']).to_frame(name='hhp_diff')
large_diff_hhp = hhp_diff[(hhp_diff['hhp_diff'] > 500) | (hhp_diff['hhp_diff'] < -500)]

# Grabbing The Data From SQL

In [16]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

In [17]:
query_all = "SELECT mgra, geotype, geozone FROM demographic_warehouse.dim.mgra WHERE series = 14 AND (geotype='cpa' OR geotype='jurisdiction')" #Remove the last and part when I do this for real 

In [18]:
sql_query = pd.read_sql_query(query_all,conn)

sql_df_all = pd.DataFrame(sql_query)

In [19]:
sql_df_all

,mgra,geotype,geozone
0,614,cpa,College Area
1,614,jurisdiction,San Diego
2,614,cpa,Mid-City:Kensington-Talmadge
3,614,jurisdiction,San Diego
4,615,cpa,Mid-City:Kensington-Talmadge
...,...,...,...
54627,6324,jurisdiction,San Diego
54628,6324,cpa,*Not in a CPA*
54629,6324,jurisdiction,Poway
54630,6325,cpa,Rancho Encantada


# Creating the CPA DF

In [20]:
SQL_CPA = sql_df_all[sql_df_all['geotype']=='cpa']
SQL_CPA = SQL_CPA.rename(columns={'geozone':'cpa_geozone'})
SQL_CPA = SQL_CPA.drop(['geotype'], axis = 1)

In [21]:
#DS38
DS38_Data_cpa = DS38_Data.merge(SQL_CPA, how='left', left_on='mgra_DS38', right_on='mgra')
DS38_Data_cpa = DS38_Data_cpa.drop(['mgra'], axis=1)
DS38_Data_cpa = DS38_Data_cpa.rename(columns={'cpa_geozone':'cpa_geozone_DS38'})

In [22]:
# DS42
DS42_Data_cpa = DS42_Data.merge(SQL_CPA, how='left', left_on='mgra_DS42', right_on='mgra')
DS42_Data_cpa = DS42_Data_cpa.drop(['mgra'], axis=1)
#DS42_Data_cpa.columns = [x + '_DS42' for x in DS42_Data_cpa.columns]

In [23]:
# Combine to make one Data Frame
cpa_df = pd.concat([DS38_Data_cpa, DS42_Data_cpa], axis=1, join='inner')


In [24]:
# Clean up CPA DF
cpa_df = cpa_df.drop(['mgra_DS38', 'cpa_geozone_DS38', 'year_DS38', 'mgra_DS42'], axis=1)
cpa_df = cpa_df.groupby(['cpa_geozone', 'year_DS42']).sum()
cpa_df = cpa_df.drop('*Not in a CPA*') #Per meeting with Purva and Mike I was instructed that I can just drop the MGRAs that do not lie in a cpa

In [25]:
# Final CPA DF
cpa_df.head(50)

pop_DS38  hhp_DS38  gq_civ_DS38  \
cpa_geozone               year_DS42                                    
32nd Street Naval Station 2016           8159       573            0   
                          2018           8503       574            0   
                          2020           9981       542            0   
                          2023          12549       845            0   
                          2025          13460      1001            0   
                          2026          13499      1040            0   
                          2029          13601      1142            0   
                          2030          13609      1150            0   
                          2032          13719      1260            0   
                          2035          13890      1431            0   
                          2040          13958      1499            0   
                          2045          14017      1558            0   
                          2050          13950      1491            0   
Alpine                    2016          20788     20533          255   
                          2018          20862     20577          285   
                          2020          20754     20469          285   
                          2023          20457     20172          285   
                          2025          20291     20006          285   
                          2026          20199     19914          285   
                          2029          19969     19684          285   
                          2030          19870     19585          285   
                          2032          19688     19403          285   
                          2035          19545     19260          285   
                          2040          19600     19315          285   
                          2045          19832     19547          285   
                          2050          19902     19617          285   
Balboa Park               2016           6296      5958          182   
                          2018           6746      6252          192   
                          2020           6855      6361          192   
                          2023           7166      6672          192   
                          2025           7446      6952          192   
                          2026           7466      6972          192   
                          2029           7690      7196          192   
                          2030           7772      7278          192   
                          2032           7761      7267          192   
                          2035           7743      7249          192   
                          2040           8162      7668          192   
                          2045           8291      7797          192   
                          2050           8367      7873          192   
Barona                    2016           2667      2666            1   
                          2018           2678      2677            1   
                          2020           2700      2699            1   
                          2023           2622      2621            1   
                          2025           2631      2630            1   
                          2026           2579      2578            1   
                          2029           2539      2538            1   
                          2030           2538      2537            1   
                          2032           2557      2556            1   
                          2035           2546      2545            1   
                          2040           2587      2586            1   

                                     gq_mil_DS38  pop_DS42  hhp_DS42  \
cpa_geozone               year_DS42                                    
32nd Street Naval Station 2016              7586      8159       573   
                          2018              7929      8509       580   
    

# Creating the Jurisdiction DF

In [26]:
SQL_jur = sql_df_all[sql_df_all['geotype']=='jurisdiction']
SQL_jur = SQL_jur.rename(columns={'geozone':'jur_geozone'})
SQL_jur = SQL_jur.drop(['geotype'], axis = 1)

In [27]:
#DS38
DS38_Data_jur = DS38_Data.merge(SQL_jur, how='left', left_on='mgra_DS38', right_on='mgra')
DS38_Data_jur = DS38_Data_jur.drop(['mgra'], axis=1)
DS38_Data_jur = DS38_Data_jur.rename(columns={'jur_geozone':'jur_geozone_DS38'})

In [28]:
#DS42
DS42_Data_jur = DS42_Data.merge(SQL_jur, how='left', left_on='mgra_DS42', right_on='mgra')
DS42_Data_jur = DS42_Data_jur.drop(['mgra'], axis=1)

In [29]:
jur_df = pd.concat([DS38_Data_jur, DS42_Data_jur], axis=1, join='inner')

In [30]:
# Ensure JurZones Are Equal, this should be zero as it is counting the number of falses  
sum(jur_df['jur_geozone_DS38'] == jur_df['jur_geozone']) - len(jur_df)

0

In [31]:
# Clean up JUR DF
jur_df = jur_df.drop(['mgra_DS38', 'jur_geozone_DS38', 'year_DS38', 'mgra_DS42'], axis=1)
jur_df = jur_df.groupby(['jur_geozone', 'year_DS42']).sum()

In [32]:
# Final Jurisdiction DF
jur_df.head(30)

pop_DS38  hhp_DS38  gq_civ_DS38  gq_mil_DS38  pop_DS42  \
jur_geozone year_DS42                                                           
Carlsbad    2016         125491    124576          915            0    125491   
            2018         127537    126622          915            0    127505   
            2020         128615    127700          915            0    128573   
            2023         128733    127818          915            0    128716   
            2025         128640    127725          915            0    128549   
            2026         128751    127836          915            0    128722   
            2029         129068    128153          915            0    129211   
            2030         129225    128310          915            0    129359   
            2032         129514    128599          915            0    129602   
            2035         131803    130888          915            0    131853   
            2040         132985    132070          915            0    133088   
            2045         133804    132889          915            0    133883   
            2050         134407    133492          915            0    134329   
Chula Vista 2016         272502    270799         1703            0    272502   
            2018         274751    273031         1720            0    274709   
            2020         274820    273100         1720            0    274808   
            2023         284890    283170         1720            0    284853   
            2025         292060    290340         1720            0    292024   
            2026         291984    290264         1720            0    291957   
            2029         291906    290186         1720            0    291991   
            2030         291972    290252         1720            0    292010   
            2032         293039    291319         1720            0    293114   
            2035         295182    293462         1720            0    295259   
            2040         307114    305394         1720            0    307080   
            2045         322452    320732         1720            0    322366   
            2050         330470    328750         1720            0    330508   
Coronado    2016          25369     18803          149         6417     25369   
            2018          25481     18915          149         6417     25477   
            2020          25522     18956          149         6417     25506   
            2023          25725     19159          149         6417     25724   

                       hhp_DS42  gq_civ_DS42  gq_mil_DS42  
jur_geozone year_DS42                                      
Carlsbad    2016         124576          915            0  
            2018         126590          915            0  
            2020         127658          915            0  
            2023         127801          915            0  
            2025         127634          915            0  
            2026         127807          915            0  
            2029         128296          915            0  
            2030         128444          915            0  
            2032         128687          915            0  
            2035         130938          915            0  
            2040         132173          915            0  
            2045         132968          915            0  
            2050         133414          915            0  
Chula Vista 2016         270799         1703            0  
            2018         272989         1720            0  
            2020         273088         1720            0  
            2023         283133         1720            0  
            2025         290304         1720            0  
            2026         290237         1720            0  
            2029         290271         1720            0  
            2030         290290         1720            0  
            2032         291394         1720      

# MGRA Analysis

In [33]:
mgra_diff_df = pd.DataFrame(index=mgra_df.index)

In [34]:
#MGRA Diff
mgra_diff_df['pop_diff'] = mgra_df['pop_DS42'] - mgra_df['pop_DS38']
mgra_diff_df['hhp_diff'] = mgra_df['hhp_DS42'] - mgra_df['hhp_DS38']
mgra_diff_df['gq_civ_diff'] = mgra_df['gq_civ_DS42'] - mgra_df['gq_civ_DS38']
mgra_diff_df['gq_mil_diff'] = mgra_df['gq_mil_DS42'] - mgra_df['gq_mil_DS38']

In [35]:
#mgra_diff_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS42_42/DS4235_results/DS4235_diff.csv', index = True)

In [36]:
# MGRA Stats 
mgra_stats = pd.DataFrame(columns=mgra_diff_df.columns, index=['Max', 'Min', 'Average'])
for col in mgra_stats.columns:
    mgra_stats[col]['Max'] = np.max(list(mgra_diff_df[col]))
    mgra_stats[col]['Min'] = np.min(list(mgra_diff_df[col]))
    mgra_stats[col]['Average'] = np.mean(list(mgra_diff_df[col]))

In [37]:
mgra_stats.head()


,pop_diff,hhp_diff,gq_civ_diff,gq_mil_diff
Max,214,214,0,0
Min,-644,-644,0,0
Average,0.0,0.0,0.0,0.0


In [38]:
#mgra_stats.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS42_42/DS4235_results/DS4235_mgra_stats.csv', index = True)

In [39]:
# Major HHP Diffs
hhp_diff = (mgra_df['hhp_DS42'] - mgra_df['hhp_DS38']).to_frame(name='hhp_diff')
large_diff_hhp = hhp_diff[(hhp_diff['hhp_diff'] > 500) | (hhp_diff['hhp_diff'] < -500)]

In [40]:
#large_diff_hhp.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS42_42/DS4235_results/large_diff_hhp.csv', index = True)

In [41]:
# Major Pop Diff
pop_diff = (mgra_df['pop_DS42'] - mgra_df['pop_DS38']).to_frame(name='pop_diff')
large_diff_pop = pop_diff[(pop_diff['pop_diff'] > 500) | (pop_diff['pop_diff'] < -500)]

In [42]:
#large_diff_pop.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS42_42/DS4235_results/large_diff_pop.csv', index = True)


# Region Analysis

In [43]:
mgra_df

pop_DS38  hhp_DS38  gq_civ_DS38  gq_mil_DS38  pop_DS42  \
mgra  year_DS42                                                           
1     2016             41        41            0            0        41   
      2018             41        41            0            0        44   
      2020             47        47            0            0        44   
      2023             38        38            0            0        46   
      2025             35        35            0            0        45   
...                   ...       ...          ...          ...       ...   
23002 2032              0         0            0            0         0   
      2035              0         0            0            0         0   
      2040              0         0            0            0         0   
      2045              0         0            0            0         0   
      2050              0         0            0            0         0   

                 hhp_DS42  gq_civ_DS42  gq_mil_DS42  
mgra  year_DS42                                      
1     2016             41            0            0  
      2018             44            0            0  
      2020             44            0            0  
      2023             46            0            0  
      2025             45            0            0  
...                   ...          ...          ...  
23002 2032              0            0            0  
      2035              0            0            0  
      2040              0            0            0  
      2045              0            0            0  
      2050              0            0            0  

[299026 rows x 8 columns]

In [44]:
mgra_names = []
for name in mgra_df.index:
    mgra_names.append(name[0])
unique_mgra_names = list(np.unique(np.array(mgra_names)))
years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']

In [45]:
mgra_sums_df = pd.DataFrame(columns=mgra_df.columns, index=years)
for col in mgra_df:
    for year in years:
            total = 0
            for mgra in unique_mgra_names:
                total += int(mgra_df.loc[mgra].loc[[year]][col])
            mgra_sums_df[col][year] = total

In [46]:
mgra_sums_df

,pop_DS38,hhp_DS38,gq_civ_DS38,gq_mil_DS38,pop_DS42,hhp_DS42,gq_civ_DS42,gq_mil_DS42
2016,3309509,3203210,63014,43285,3309509,3203210,63014,43285
2018,3353437,3237268,69372,46797,3353437,3237268,69372,46797
2020,3383955,3263592,72056,48307,3383955,3263592,72056,48307
2023,3434113,3309666,73875,50572,3434113,3309666,73875,50572
2025,3470849,3345075,74447,51327,3470849,3345075,74447,51327
2026,3488258,3362198,74733,51327,3488258,3362198,74733,51327
2029,3537297,3410379,75591,51327,3537297,3410379,75591,51327
2030,3552486,3425282,75877,51327,3552486,3425282,75877,51327
2032,3580821,3453451,76043,51327,3580821,3453451,76043,51327
2035,3620349,3492730,76292,51327,3620349,3492730,76292,51327


In [ ]:
C:\Users\cra\OneDrive - San Diego Association of Governments\DS41_42\DS4238_v2_results

In [47]:
mgra_sums_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/DS4238_v2_results/MGRA/DS4238_v2_mgra_sums.csv', index = True)

# CPA Analysis

In [100]:
cpa_df.head()

pop_DS35  hhp_DS35  gq_civ_DS35  \
cpa_geozone               year_DS41                                    
32nd Street Naval Station 2016           8159       573            0   
                          2018           8513       584            0   
                          2020           9992       553            0   
                          2023          12272       568            0   
                          2025          12998       539            0   

                                     gq_mil_DS35  pop_DS41  hhp_DS41  \
cpa_geozone               year_DS41                                    
32nd Street Naval Station 2016              7586      8159       573   
                          2018              7929      8492       563   
                          2020              9439     10005       566   
                          2023             11704     12255       551   
                          2025             12459     13019       560   

                                     gq_civ_DS41  gq_mil_DS41  
cpa_geozone               year_DS41                            
32nd Street Naval Station 2016                 0         7586  
                          2018                 0         7929  
                          2020                 0         9439  
                          2023                 0        11704  
                          2025                 0        12459

In [104]:
cpa_diff_df = pd.DataFrame(index=cpa_df.index)

In [105]:
#41 minus 35
cpa_diff_df['pop_diff'] = cpa_df['pop_DS42'] - cpa_df['pop_DS38']
cpa_diff_df['hhp_diff'] = cpa_df['hhp_DS42'] - cpa_df['hhp_DS38']
cpa_diff_df['gq_civ_diff'] = cpa_df['gq_civ_DS42'] - cpa_df['gq_civ_DS38']
cpa_diff_df['gq_mil_diff'] = cpa_df['gq_mil_DS42'] - cpa_df['gq_mil_DS38']

In [107]:
cpa_diff_df.to_csv(r'C:/Users/cra/OneDrive - San Diego Association of Governments/DS42_42/DS4235_results/CPA_DS4235_diff.csv', index = True)

In [106]:
cpa_diff_df.head(10)

pop_diff  hhp_diff  gq_civ_diff  \
cpa_geozone               year_DS41                                    
32nd Street Naval Station 2016              0         0            0   
                          2018            -21       -21            0   
                          2020             13        13            0   
                          2023            -17       -17            0   
                          2025             21        21            0   
                          2026             19        19            0   
                          2029             -2        -2            0   
                          2030             -7        -7            0   
                          2032            -10       -10            0   
                          2035            -27       -27            0   

                                     gq_mil_diff  
cpa_geozone               year_DS41               
32nd Street Naval Station 2016                 0  
                          2018                 0  
                          2020                 0  
                          2023                 0  
                          2025                 0  
                          2026                 0  
                          2029                 0  
                          2030                 0  
                          2032                 0  
                          2035                 0

In [59]:
# 41 minus 35
cpa_stats = pd.DataFrame(columns=cpa_diff_df.columns, index=['Max', 'Min', 'Average'])
for col in mgra_stats.columns:
    cpa_stats[col]['Max'] = np.max(list(cpa_diff_df[col]))
    cpa_stats[col]['Min'] = np.min(list(cpa_diff_df[col]))
    cpa_stats[col]['Average'] = np.mean(list(cpa_diff_df[col]))

In [60]:
cpa_stats

,pop_diff,hhp_diff,gq_civ_diff,gq_mil_diff
Max,1985,1985,0,0
Min,-1112,-1112,0,0
Average,5.777098,5.777098,0.0,0.0


## CPA Sums

In [61]:
CPA_names = []
for name in cpa_df.index:
    CPA_names.append(name[0])
unique_cpa_names = list(np.unique(np.array(CPA_names)))
years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']

In [62]:
cpa_sums_df = pd.DataFrame(columns=cpa_df.columns, index=years)
for col in cpa_df:
    for year in years:
            total = 0
            for cpa in unique_cpa_names:
                total += int(cpa_df.loc[cpa].loc[[year]][col])
            cpa_sums_df[col][year] = total


In [63]:
cpa_sums_df

,pop_DS35,hhp_DS35,gq_civ_DS35,gq_mil_DS35,pop_DS41,hhp_DS41,gq_civ_DS41,gq_mil_DS41
2016,3906368,3732674,77690,96004,3906368,3732674,77690,96004
2018,3960441,3772343,84651,103447,3959547,3771449,84651,103447
2020,4003343,3805479,88377,109487,4003080,3805216,88377,109487
2023,4076107,3866589,90971,118547,4075581,3866063,90971,118547
2025,4127648,3914538,91543,121567,4127813,3914703,91543,121567
2026,4151940,3938544,91829,121567,4151861,3938465,91829,121567
2029,4214983,4000729,92687,121567,4214434,4000180,92687,121567
2030,4235273,4020733,92973,121567,4236157,4021617,92973,121567
2032,4270172,4055466,93139,121567,4272261,4057555,93139,121567
2035,4320952,4105997,93388,121567,4323047,4108092,93388,121567


# Check the Jurisdiction Level

In [64]:
jur_diff_df = pd.DataFrame(index=jur_df.index)

In [65]:
#41 minus 35
jur_diff_df['pop_diff'] = jur_df['pop_DS42'] - jur_df['pop_DS38']
jur_diff_df['hhp_diff'] = jur_df['hhp_DS42'] - jur_df['hhp_DS38']
jur_diff_df['gq_civ_diff'] = jur_df['gq_civ_DS42'] - jur_df['gq_civ_DS38']
jur_diff_df['gq_mil_diff'] = jur_df['gq_mil_DS42'] - jur_df['gq_mil_DS38']

In [66]:
# 41 minus 35
jur_stats = pd.DataFrame(columns=jur_diff_df.columns, index=['Max', 'Min', 'Average'])
for col in jur_stats.columns:
    jur_stats[col]['Max'] = np.max(list(jur_diff_df[col]))
    jur_stats[col]['Min'] = np.min(list(jur_diff_df[col]))
    jur_stats[col]['Average'] = np.mean(list(jur_diff_df[col]))
jur_stats

,pop_diff,hhp_diff,gq_civ_diff,gq_mil_diff
Max,2409,2409,0,0
Min,-960,-960,0,0
Average,26.757085,26.757085,0.0,0.0


# Checking the sums

In [67]:
jur_names = []
for name in jur_df.index:
    jur_names.append(name[0])
unique_jur_names = list(np.unique(np.array(jur_names)))
years = ['2016','2018','2020','2023','2025','2026','2029','2030','2032','2035','2040','2045','2050']

In [68]:
jur_sums_df = pd.DataFrame(columns=jur_df.columns, index=years)
for col in jur_df:
    for year in years:
            total = 0
            for jur in unique_jur_names:
                total += int(jur_df.loc[jur].loc[[year]][col])
            jur_sums_df[col][year] = total

In [69]:
jur_sums_df

,pop_DS35,hhp_DS35,gq_civ_DS35,gq_mil_DS35,pop_DS41,hhp_DS41,gq_civ_DS41,gq_mil_DS41
2016,3906368,3732674,77690,96004,3906368,3732674,77690,96004
2018,3960441,3772343,84651,103447,3959547,3771449,84651,103447
2020,4003343,3805479,88377,109487,4003080,3805216,88377,109487
2023,4076107,3866589,90971,118547,4075581,3866063,90971,118547
2025,4127648,3914538,91543,121567,4127813,3914703,91543,121567
2026,4151940,3938544,91829,121567,4151861,3938465,91829,121567
2029,4214983,4000729,92687,121567,4214434,4000180,92687,121567
2030,4235273,4020733,92973,121567,4236157,4021617,92973,121567
2032,4270172,4055466,93139,121567,4272261,4057555,93139,121567
2035,4320952,4105997,93388,121567,4323047,4108092,93388,121567
